## Azure Machine Learning SDK for Python

Sources from __[What is the Azure Machine Learning SDK for Python?](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py)__

Key areas of the SDK include:

- Explore, prepare and manage the lifecycle of your datasets used in machine learning experiments.
- Manage cloud resources for monitoring, logging, and organizing your machine learning experiments.
- Train models either locally or by using cloud resources, including GPU-accelerated model training.
- Use automated machine learning, which accepts configuration parameters and training data. It automatically iterates through algorithms and hyperparameter settings to find the best model for running predictions.
- Deploy web services to convert your trained models into RESTful services that can be consumed in any application.


AML SDK for Python Namespace:
* Workspace
* Dataset
* Experiment
* Run
* Model
* ComputeTarget
* RunConfiguration
* ScriptRunConfig
* Environment
* Pipeline
* PythonScriptStep

![AMLWorkspace](https://docs.microsoft.com/en-us/azure/machine-learning/media/concept-workspace/azure-machine-learning-taxonomy.png#lightbox)


## Check version of the SDK

In [ ]:
import azureml.core


print("Azure Machine Learning SDK for python version {0}".format(azureml.core.VERSION))

## Workspace

### azureml.core.workspace.Workspace

Create AML Workspace

```python
from azureml.core import Workspace
ws = Workspace.create(name='myworkspace',
           subscription_id='<azure-subscription-id>',
           resource_group='myresourcegroup',
           create_resource_group=True,
           location='eastus2'
           )
```

__Get workspace object__

```python
from azureml.core import Workspace

ws = Workspace.get(name="myworkspace",
            subscription_id='<azure-subscription-id>',
            resource_group='myresourcegroup')
```

In [ ]:
from azureml.core import Workspace


# Read from '.azureml/' in the current working directory and 'config.json' file
ws = Workspace.from_config()

ws.get_details()

In [ ]:
fs = open('./.azureml/config.json')

print(fs.read())

## Datastore and DataSet


In [ ]:
csvfile = 'diabetes.csv'

In [ ]:
from azureml.core import Datastore

datastore_name = 'workspaceblobstore' # Update the value with your datastore name

# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(ws, datastore_name)
datastore

In [ ]:
file_datastore = datastore.upload('./data', 'data', overwrite=True)

In [ ]:
from azureml.core import Dataset

ds = Dataset.File.from_files(path=[(datastore,'./data/diabetes.csv')])
ds.register(ws, 'diabetes', create_new_version=True)

## Experiment

### azureml.core.experiment.Experiment

The Experiment class is another foundational cloud resource that represents a collection of trials (individual model runs). The following code fetches an Experiment object from within Workspace by name, or it creates a new Experiment object if the name doesn't exist.


In [ ]:
from azureml.core.experiment import Experiment

expName = "mtc-aml-lab-exp"
exp = Experiment(workspace=ws, name=expName)
exp

In [ ]:
exp.tag("projectName","AML-Lab")
exp.tag("MTCLocation","Seattle")
exp.tag("MTCTeam","MTCS")
exp.tag("MTCTeam","MTC Seattle") # Careful, tags are mutable

In [ ]:
list_experiments = Experiment.list(ws)
list_experiments

In [ ]:
for experiment in list_experiments:
    if experiment.name == expName:
        print(experiment.name) 
        print(experiment.tags)

# Check the value of key 'Team'

In [ ]:
list_runs = exp.get_runs()

for run in list_runs:
    print(run.id)

There are two ways to execute an experiment trial. 

If you're interactively experimenting in a Jupyter notebook, use the `start_logging` function. 

If you're submitting an experiment from a standard Python environment, use the `submit` function. 

Both functions return a Run object. The experiment variable represents an Experiment object in the following code examples.

In [ ]:
run = exp.start_logging()
run

In [ ]:
list_runs = exp.get_runs()

for run in list_runs:
    print(run)

## Run

### azureml.core.run.Run

A run represents a single trial of an experiment. Run is the object that you use to monitor the asynchronous execution of a trial, store the output of the trial, analyze results, and access generated artifacts. You use Run inside your experimentation code to log metrics and artifacts to the Run History service. Functionality includes:

- Storing and retrieving metrics and data.
- Using tags and the child hierarchy for easy lookup of past runs.
- Registering stored model files for deployment.
- Storing, modifying, and retrieving properties of a run.

Create a Run object by submitting an Experiment object with a run configuration object. Use the tags parameter to attach custom categories and labels to your runs. You can easily find and retrieve them later from Experiment.


In [ ]:
# run = experiment.submit(config=your_config_object, tags=tags)

run.tag("owner","hyun")
run.tag("build","dev")
run.tag("codeVersion",1) # Integer or string for value

In [ ]:
print(run)

In [ ]:
from azureml.core.run import Run

filtered_list_runs = Run.list(exp, tags={"owner":"hyun", "build":"dev"})

for filtered_run in filtered_list_runs:
    print(filtered_run)
    print(filtered_run.tags)

In [ ]:
run_details = run.get_details()
run_details

Output for this function is a dictionary that includes:

- Run ID
- Status
- Start and end time
- Compute target (local versus cloud)
- Dependencies and versions used in the run
- Training-specific data (differs depending on model type)

### Upload file/s to AML using RUN

In [ ]:
run.upload_file(name='aml-lab/workshop.ipynb', path_or_stream="./0.AMLSDKforPython.ipynb")

# Go to 'Run' > 'Outputs + Logs' in Experiment of Azure Machine Learning

### Logging metrics using RUN

In [ ]:
run.log_list(name='Fibonacci', value=[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89])

In [ ]:
import numpy as np


for i in (range(-10, 10)): 
    run.log(name='Sigmoid', value=1 / (1 + np.exp(-i)))
    angle = i / 2.0

In [ ]:
for i in (range(-10, 10)):
    angle = i / 2.0
    run.log_row(name='Cosine Wave', angle=angle, cos=np.cos(angle))


In [ ]:
citrus = ['orange', 'lemon', 'lime']
sizes = [ 10, 7, 3]

for index in range(len(citrus)):
    run.log_row("citrus", fruit = citrus[index], size=sizes[index])

In [ ]:
run.log_image(name='AML Concept Whiteboard', path='./AML_concept.png', plot=None, description='Discussion lead by Hyun at MTC Seattle')

In [ ]:
metrics = run.get_metrics()
# metrics is of type Dict[str, List[float]] mapping metric names
# to a list of the values for that metric in the given run.

print(metrics)

In [ ]:
run.complete()
# run.cancel()

## Model

### azureml.core.model.Model

The `Model` class is used for working with cloud representations of machine learning models. Methods help you transfer models between local development environments and the `Workspace` object in the cloud.

You can use model registration to store and version your models in the Azure cloud, in your workspace. Registered models are identified by name and version. Each time you register a model with the same name as an existing one, the registry increments the version. Azure Machine Learning supports any model that can be loaded through Python 3, not just Azure Machine Learning models.

The following example shows how to build a simple local classification model with `scikit-learn`, register the model in `Workspace`, and download the model from the cloud.

Create a simple classifier, `clf`, to predict customer churn based on their age. Then dump the model to a `.pkl` file in the same directory.

In [ ]:
from sklearn import svm
import joblib
import numpy as np

# customer ages
X_train = np.array([50, 17, 35, 23, 28, 40, 31, 29, 19, 62])
X_train = X_train.reshape(-1, 1)
# churn y/n
y_train = ["yes", "no", "no", "no", "yes", "yes", "yes", "no", "no", "yes"]

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(X_train, y_train)

joblib.dump(value=clf, filename="churn-model.pkl")

Use the `register` function to register the model in your workspace. Specify the local model path and the model name. Registering the same name more than once will create a new version.

In [ ]:
from azureml.core.model import Model

model = Model.register(workspace=ws,
                       model_path="churn-model.pkl",
                       model_name="churn-model-test")

Now that the model is registered in your workspace, it's easy to manage, download, and organize your models. To retrieve a model (for example, in another environment) object from `Workspace`, use the class constructor and specify the model name and any optional parameters. Then, use the download function to `download` the model, including the cloud folder structure.

## Environment

### azureml.core.environment.Environment

Azure Machine Learning environments specify the Python packages, environment variables, and software settings around your training and scoring scripts. In addition to Python, you can also configure PySpark, Docker and R for environments. Internally, environments result in Docker images that are used to run the training and scoring processes on the compute target. The environments are managed and versioned entities within your Machine Learning workspace that enable reproducible, auditable, and portable machine learning workflows across a variety of compute targets and compute types.

You can use an Environment object to:

- Develop your training script.
- Reuse the same environment on Azure Machine Learning Compute for model training at scale.
- Deploy your model with that same environment without being tied to a specific compute type.

The following code imports the Environment class from the SDK and to instantiates an environment object.

In [ ]:
import sys
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# pyVersion = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
computeInstanceName = '' # <update the value with your Compute Instance Name>

myenv = Environment(name="myEnv")
conda_dep = CondaDependencies()

# Installs pillow package
conda_dep.add_conda_package("numpy==1.17.0")
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("pillow")
conda_dep.add_pip_package("azureml-dataprep[pandas]")


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep
myenv.python.conda_dependencies.set_python_version("3.6")
myenv.register(ws)
myenv.build(ws, computeInstanceName)

In [ ]:
from azureml.core.model import Model
import os

model = Model(workspace=ws, name="churn-model-test")
model.download(target_dir=os.path.join(os.getcwd(),"myDownload"))

## ComputeTarget

### azureml.core.compute.ComputeTarget

The `ComputeTarget` class is the abstract parent class for creating and managing compute targets. A compute target represents a variety of resources where you can train your machine learning models. A compute target can be either a local machine or a cloud resource, such as Azure Machine Learning Compute, Azure HDInsight, or a remote virtual machine.

Use compute targets to take advantage of powerful virtual machines for model training, and set up either persistent compute targets or temporary runtime-invoked targets. For a comprehensive guide on setting up and managing compute targets, see the how-to.

The following code shows a simple example of setting up an `ComputeInstance` target. The resource scales automatically when a job is submitted. It's deleted automatically when the run finishes.

Reuse the simple scikit-learn churn model and build it into its own file, train.py, in the current directory. At the end of the file, create a new directory called outputs. This step creates a directory in the cloud (your workspace) to store your trained model that joblib.dump() serialized.

In [ ]:
from azureml.core.compute import ComputeInstance 

computeInstanceName = ''
myInstance = ComputeInstance(ws, computeInstanceName)

## RunConfiguration

### azureml.core.compute.RunConfiguration

Next you create the compute target by instantiating a RunConfiguration object and setting the type and size. This example uses the smallest resource size (1 CPU core, 3.5 GB of memory). The list_vms variable contains a list of supported virtual machines and their sizes.

In [ ]:
from azureml.core.runconfig import RunConfiguration

compute_config = RunConfiguration()
compute_config.target = myInstance
compute_config

Define Environment

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")

compute_config.environment.python.conda_dependencies = dependencies
compute_config.environment = myenv

In [ ]:
%%writefile ./source/train/train.py
from sklearn import svm
import numpy as np
import joblib
import os

# customer ages
X_train = np.array([50, 17, 35, 23, 28, 40, 31, 29, 19, 62])
X_train = X_train.reshape(-1, 1)
# churn y/n
y_train = ["yes", "no", "no", "no", "yes", "yes", "yes", "no", "no", "yes"]

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(X_train, y_train)

os.makedirs("outputs", exist_ok=True)
joblib.dump(value=clf, filename="outputs/churn-model.pkl")

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(
    source_directory="./source/train",
    script="train.py",
    run_config=compute_config)

experiment = Experiment(workspace=ws, name=expName)

run = experiment.submit(config=script_run_config)

# This may take around 7 minutes
run.wait_for_completion(show_output=True)

> TIP
> [Prep your code for production](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-convert-ml-experiment-to-production)

## ScriptRunConfig

### azureml.core.script_run_config.ScriptRunConfig


In [ ]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

runconfig = ScriptRunConfig(
    source_directory="./source/train",
    script="train.py")

# Attach compute target to run config
runconfig.run_config.target = "local"

# Attach environment to run config
runconfig.run_config.environment = myenv

experiment = Experiment(workspace=ws, name=expName)

run = experiment.submit(config=script_run_config)

run.wait_for_completion(show_output=True)

## Pipeline, PythonScriptStep

### azureml.pipeline.core.pipeline.Pipeline
### azureml.pipeline.steps.python_script_step.PythonScriptStep


An Azure Machine Learning pipeline is an automated workflow of a complete machine learning task. Subtasks are encapsulated as a series of steps within the pipeline. An Azure Machine Learning pipeline can be as simple as one step that calls a Python script. Pipelines include functionality for:

- Data preparation including importing, validating and cleaning, munging and transformation, normalization, and staging
- Training configuration including parameterizing arguments, filepaths, and logging / reporting configurations
- Training and validating efficiently and repeatably, which might include specifying specific data subsets, different hardware compute resources, distributed processing, and progress monitoring
- Deployment, including versioning, scaling, provisioning, and access control
- Publishing a pipeline to a REST endpoint to rerun from any HTTP library

A ```PythonScriptStep``` is a basic, built-in step to run a Python Script on a compute target. It takes a script name and other optional parameters like arguments for the script, compute target, inputs and outputs. 

### Pattern for creating and using ML Pipeline

An Azure Machine Learning pipeline is associated with an Azure Machine Learning workspace and a pipeline step is associated with a compute target available within that workspace. For more information, see this article about workspaces or this explanation of compute targets.

A common pattern for pipeline steps is:

1. Specify workspace, compute, and storage
2. Configure your input and output data using
    1. Dataset which makes available an existing Azure datastore
    2. PipelineDataset which encapsulates typed tabular data
    3. PipelineData which is used for intermediate file or directory data written by one step and intended to be consumed by another
3. Define one or more pipeline steps
4. Instantiate a pipeline using your workspace and steps
5. Create an experiment to which you submit the pipeline
6. Monitor the experiment results

In [ ]:
from azureml.core.dataset import Dataset

web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path)
titanic_ds.register(ws, 'titanic', create_new_version=True)

In [ ]:
%%writefile ./source/prepdata/prepare.py

import os
import argparse
from azureml.core import Dataset, Run

parser = argparse.ArgumentParser("prep")

parser.add_argument("--output_path", type=str, help="output data directory")

args = parser.parse_args()

print(os.environ)

run = Run.get_context()
ws = run.experiment.workspace

dataset = run.input_datasets['titanic']
df = dataset.to_pandas_dataframe()
df = df.drop(labels=['PassengerId', 'Name', 'Ticket', 'Fare', 'Embarked'], axis=1)

print(df)

os.makedirs(args.output_path)
df.to_csv(os.path.join(args.output_path, 'prepdTitanic.csv'))

In [ ]:
from azureml.core import Dataset, RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData

# get input dataset
input_ds = Dataset.get_by_name(ws, 'titanic')

# register pipeline output as dataset
output_ds = PipelineData('prepared_titanic_ds', datastore=datastore).as_dataset()
output_ds = output_ds.register(name='prepared_titanic', create_new_version=True)

# configure for runconfig
runconfig = RunConfiguration()
runconfig.environment = myenv

# configure pipeline step to use dataset as the input and output
prep_step = PythonScriptStep(script_name="prepare.py",
                             arguments=["--output_path",output_ds],
                             inputs=[input_ds.as_named_input('titanic')],
                             outputs=[output_ds],
                             compute_target=compute_target,
                             source_directory="./source/prepdata",
                             runconfig=runconfig)


In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[prep_step])
pipeline_run = exp.submit(pipeline)

In [ ]:
#End of notebook